In [ ]:
import numpy as np
import keras
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
# Crear un dataset con todas las imágenes de un directorio y escalarlas al ancho y alto necesario para la arquitectura elegida

# Parámetros del notebook
path = "/content/test"     # El directorio donde se leeran las imágenes
                           # El directorio a leer deberá contar con subdirectorios, uno para cada clase
                           # que se desee validar
                           # Ejemplo:
                           # ./Test
                           #     |------ Clase1
                           #     |           |------ Imagen1.jpg
                           #     |           |------ Imagen2.jpg
                           #     |           |------ Imagen3.jpg
                           #     |------ Clase2
                           #                 |------ Imagen4.jpg
                           #                 |------ Imagen5.jpg
                           #                 |------ Imagen6.jpg

# 224x244x3 (RGB) son las dimensiones que espera VGG16
width = 224   # ancho de las imágenes necesario para la arquitectura elegida
height = 224  # alto de las imágenes necesario para la arquitectura elegida

# Función de preprocesamiento de las imágenes (propia de cada arquitectura)
preprocessing_function = keras.applications.vgg16.preprocess_input


In [ ]:
# Crear el "cargador" de imágenes
datagen = ImageDataGenerator(preprocessing_function=preprocessing_function)

generator = datagen.flow_from_directory(path, target_size=(height, width))

# Obtener el listado de los archivos leidos
file_names = generator.filenames

# Cargar el nombre de las clases de ImageNet
content = open("ImageNet.classes").readlines()
class_names = [line[9:].strip() for line in content]

In [ ]:
# Cargar el modelo preentrenado, puede usar cualquier arquitectura vista en clase, otra provista por keras o alguna otra creada por usted

model = keras.applications.VGG16(
    include_top=True,
    weights="imagenet",
    input_tensor=None,
    input_shape=None,
    pooling=None,
    classes=1000,
    classifier_activation="softmax",
)
model.summary()
keras.utils.plot_model(model)

In [ ]:
# Validación del conjunto de testeo

# Usar el modelo cargado para hacer la predicción
generator.reset()
results = model.predict(generator)

# Buscar la clase con valor más alto para cada muestra
preds = np.argmax(results, axis=1)

# Impresión de resultados
print('Predicted class\tProb. of class\t(filename)')
for i, p in enumerate(preds):
    pred_index = p
    pred_class = class_names[pred_index]
    file_name = file_names[i]
    prob = results[i][pred_index]
    print(f'{pred_class}\t{prob}\t({file_name})')
